In [1]:
import matplotlib.pyplot as plt

import numpy as np

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Model
from keras import optimizers
from keras import regularizers

In [2]:
def tvscale(img):
  plt.rcParams['figure.figsize'] = [6,6]
  imgplot = plt.imshow(img, interpolation='none', cmap='gray')
  plt.axis('off')
  plt.show()

In [4]:
try:
  X1 = np.load('./cosmo-late.npy')
except:
  !wget https://github.com/Morisset/Python-lectures-Notebooks/raw/master/Notebooks/cosmo-early.npy
  X1 = np.load('./cosmo-late.npy')
try:
  X2 = np.load('./cosmo-early.npy')
except:
  !wget https://github.com/Morisset/Python-lectures-Notebooks/raw/master/Notebooks/cosmo-late.npy
  X2 = np.load('./cosmo-early.npy')

In [5]:
#tvscale(X1[0])

In [22]:
#-- Definition of the Neural Network
input_img = Input(shape=(128, 128, 3))

activ = 'tanh'

x = Conv2D(filters=16, kernel_size=(3,3), activation=activ, padding='same')(input_img) #-- 128, 128, 3 -> 128, 128, 16
x = MaxPooling2D(pool_size=(2,2), padding='same')(x) #-- 128, 128, 16 -> 64, 64, 16

x = Conv2D(filters=32, kernel_size=(3,3), activation=activ, padding='same')(x) #-- 64, 64, 16 -> 64, 64, 32
x = MaxPooling2D(pool_size=(2,2), padding='same')(x) #-- 64, 64, 32 -> 32, 32, 32

x = Conv2D(filters=64, kernel_size=(3,3), activation=activ, padding='same')(x) #-- 32, 32, 32 -> 32, 32, 64
x = MaxPooling2D(pool_size=(2,2), padding='same')(x) #-- 32, 32, 64 -> 16, 16, 64

x = Conv2D(filters=128, kernel_size=(3,3), activation=activ, padding='same')(x) #-- 
x = MaxPooling2D(pool_size=(2,2), padding='same')(x) #-- 

x = Conv2D(filters=256, kernel_size=(3,3), activation=activ, padding='same')(x) #-- 
x = MaxPooling2D(pool_size=(2,2), padding='same')(x) #-- 

x = Flatten()(x) #-- 16, 16, 64 -> 16,384
x = Dense(32, activation=activ)(x)
x = Dense(16, activation=activ)(x)
x = Dropout(0.3)(x)

out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_img, outputs=out)
model.compile(loss='mse', optimizer='adam', metrics='mse')

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 16, 16, 64)        0   

In [23]:
#-- Create training sets

X = np.concatenate((X1, X2), axis=0) / 256.
Y = np.zeros(X.shape[0])
Y[0:len(X1)] = 1

rand_ind = np.arange(0, X.shape[0])

np.random.seed(42)
np.random.shuffle(rand_ind)

X = X[rand_ind]
Y = Y[rand_ind]


In [24]:
X_train = X[50:]
Y_train = Y[50:]

X_test = X[0:50]
Y_test = Y[0:50]

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(552, 128, 128, 3) (552,) (50, 128, 128, 3) (50,)


In [25]:
history = model.fit(X_train, Y_train, batch_size=32, epochs=100, shuffle=True)

Epoch 1/100
18/18 [==============================] - 1s 17ms/step - loss: 0.2409 - mse: 0.2409
Epoch 2/100
18/18 [==============================] - 0s 13ms/step - loss: 0.0760 - mse: 0.0760
Epoch 3/100
18/18 [==============================] - 0s 13ms/step - loss: 0.0703 - mse: 0.0703
Epoch 4/100
18/18 [==============================] - 0s 12ms/step - loss: 0.0328 - mse: 0.0328
Epoch 5/100
18/18 [==============================] - 0s 13ms/step - loss: 0.0176 - mse: 0.0176
Epoch 6/100
18/18 [==============================] - 0s 12ms/step - loss: 0.0163 - mse: 0.0163
Epoch 7/100
18/18 [==============================] - 0s 13ms/step - loss: 0.0207 - mse: 0.0207
Epoch 8/100
18/18 [==============================] - 0s 12ms/step - loss: 0.0202 - mse: 0.0202
Epoch 9/100
18/18 [==============================] - 0s 13ms/step - loss: 0.0048 - mse: 0.0048
Epoch 10/100
18/18 [==============================] - 0s 13ms/step - loss: 0.0137 - mse: 0.0137
Epoch 11/100
18/18 [=============================

In [26]:
Y_pred = np.round(model.predict(X_test))[:,0]

equal = np.where(Y_pred - Y_test == 0)[0]

print(len(equal) / 50)

0.98
